In [ ]:
!pip install -qq openai
import openai
import os
import re

In [ ]:
github_token = "GITHUB TOKEN"
!git clone https://$github_token@github.com/fsame/gpt_humeval.git


In [ ]:
%cd gpt_humeval


In [ ]:
!git status
!git pull origin main  # or the branch you are working with

In [ ]:
openai.api_key = "API KEY"

In [ ]:
def replace_refs(text, helper_sentence):
    messages = [
        {"role": "system", "content": "Generate natural referring expressions for specific referents marked in the text."},
        {"role": "user", "content": f"""
        subject: In the following, you will be presented with a news text from an American newspaper.
        The text will talk about one specific subject, which can be, for example, a person, a company, a group, a country or a commercial product.
        In the text, all references to one subject are replaced by [REF_number_number] (e.g., REF_1000_1, REF_20_5, etc).
        Your goal is to fill those gaps, referring to the main subject of the text, so that the text becomes easy to understand.
Always talk about the whole subject: so if the topic is *Mr and Mrs Smith* and you know they are a couple from London, then you can fill a box with "Mr and Mrs Smith", "the London couple", "this couple", "they", "Mr and Mrs Smith's", "the couple's" and so on. But a box is never about "Mr Smith" or "Mrs Smith" on their own.
Although we show you words like "he", "she", "it", "they" or similar, you do not have to use them. In the box you can put any way of identifying the subject. If the subject is "Joe Biden, the president of the United States" you can put "Joe Biden" or "Mr Biden" or "Joe Biden's" or "the president of the United States" or "the president" or "the president's" or "him" or "his", or any other way of identifying this person.
At the start of each text, you will see a line which gives you the subject of the text (e.g., subject: Margaret Thatcher (she/her)), and the thing or person you need to refer to as you fill out the boxes. To help you understand how to talk about the thing or person you will also have a helper sentence providing more information (e.g., helper sentence: Margaret Thatcher was the prime minister of the United Kingdom.). IMPORTANT POINT: No need to rewrite the whole text. Just write the expressions that are generated as follows:
REF_1000_1:
REF_20_5:
etc.
:\n
subject: {subject_line} \n
helper sentence: {helper_sentence} \n
text: {text}"""}
    ]
    response = openai.ChatCompletion.create(
        #previous model: gpt-3.5-turbo
        model="gpt-4",
        messages=messages
    )
    return response['choices'][0]['message']['content'].strip()

# Specify the folder containing the .txt files
folder_path = "../gpt_humeval/GPT_experiment/lists/list5"
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

#Open an output file to write the results
output_file = open("../list5rep19.txt", "w")

# Loop through each .txt file and process it
for txt_file in txt_files:
    file_path = os.path.join(folder_path, txt_file)
    with open(file_path, "r") as f:
        lines = f.readlines()

    subject_line = lines[0].strip().split(": ")[1]
    helper_sentence = lines[1].strip().split(": ")[1]


    # Concatenate all lines that come after 'text:'
    text_start_index = next(i for i, line in enumerate(lines) if line.startswith("text:"))
    text = " ".join(line.strip() for line in lines[text_start_index:]).split(": ", 1)[1]

    # Replace the placeholders in the text
    new_text = replace_refs(text, helper_sentence)
    #print(new_text)

    # Write the results to the output file
    output_file.write(new_text + "\n")

    # Close the output file
output_file.close()